#### IMPORTO LIBRERIAS ####

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model

#configuro notebook para mostrar varios salidas en la misma celda
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

!pwd
!ls -l

/Users/mac/Documents/MASTER_DATA_SCIENCE/tfm
total 696
-rw-r--r--   1 mac  staff  104803 15 abr 11:08 TFM_DGT-pruebas.ipynb
-rw-r--r--   1 mac  staff  246127 24 may 12:06 TFM_DGT.ipynb
drwxr-xr-x  11 mac  staff     374 23 may 23:03 cleaned
-rw-r--r--   1 mac  staff       0 23 may 22:42 gg
drwxr-xr-x   5 mac  staff     170 30 mar 23:28 metadata
drwxr-xr-x  11 mac  staff     374 30 mar 23:28 raw
drwxr-xr-x  14 mac  staff     476 11 may 17:18 raw_zip


### CONSERVO LOS DATOS ORIGINALES EN LA CARPETA RAW. LOS FICHEROS DE TRABAJO ESTAN EN LA CARPETA CLEANED

In [2]:
!ls -l
!cp -r raw/* cleaned/

total 696
-rw-r--r--   1 mac  staff  104803 15 abr 11:08 TFM_DGT-pruebas.ipynb
-rw-r--r--   1 mac  staff  246127 24 may 12:06 TFM_DGT.ipynb
drwxr-xr-x  11 mac  staff     374 23 may 23:03 cleaned
-rw-r--r--   1 mac  staff       0 23 may 22:42 gg
drwxr-xr-x   5 mac  staff     170 30 mar 23:28 metadata
drwxr-xr-x  11 mac  staff     374 30 mar 23:28 raw
drwxr-xr-x  14 mac  staff     476 11 may 17:18 raw_zip


### GUARDO UN LISTADO POR TIPOLOGIA DE FICHERO, PARA POSTERIORMENTE ALMACENARLO EN UN DICCIONARIO ###

In [3]:
accvict=!ls ./cleaned/MICRODATOS_ACC_VICT_20*/TABLA_ACCVICT_*.csv
pers=!ls ./cleaned/MICRODATOS_ACC_VICT_20*/TABLA_PERS_*.csv
vehic=!ls ./cleaned/MICRODATOS_ACC_VICT_20*/TABLA_VEHIC_*.csv
dic_all_files={'Accvict':accvict,'Pers':pers,'Vehic':vehic}
len(dic_all_files)

3

#### RUTAS DE TODOS LOS FICHEROS ####

In [4]:
dic_all_files

{'Accvict': ['./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv',
  './cleaned/MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv',
  './cleaned/MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv',
  './cleaned/MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv',
  './cleaned/MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv',
  './cleaned/MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv',
  './cleaned/MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv',
  './cleaned/MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv'],
 'Pers': ['./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_PERS_24H_2008.csv',
  './cleaned/MICRODATOS_ACC_VICT_2009/TABLA_PERS_24H_2009.csv',
  './cleaned/MICRODATOS_ACC_VICT_2010/TABLA_PERS_24H_2010.csv',
  './cleaned/MICRODATOS_ACC_VICT_2011/TABLA_PERS_2011.csv',
  './cleaned/MICRODATOS_ACC_VICT_2012/TABLA_PERS_2012.csv',
  './cleaned/MICRODATOS_ACC_VICT_2013/TABLA_PERS_2013.csv',
  './cleaned/MICRODATOS_ACC_VICT_2014/TABLA_PERS_2014.csv',
  './cleaned/MICRODATOS_ACC_VICT

### Ahora que los datos estan trazados, procedemos a realizar una observacion preliminar para comprobar si hay homgeneidad en los datos, y si disponemos de la misma informacion en cada data set, ya que segun la DGT, las extruturas de los datos cambian a partir del 2011 en adelante.

### Recorro cada clave - para obtener las cabeceras y poder comparar que campos tengo en cada año por tipo de fichero, e introduzco los data frames en una lista.

In [5]:
dic_all_files.keys()

dict_keys(['Accvict', 'Pers', 'Vehic'])

In [6]:
Headers_List=[]
for key in dic_all_files.keys():
    for year in [0,7]:
        if year==0:
            df_2008=pd.read_csv(dic_all_files[key][year],delimiter=';',nrows=0,encoding="ISO-8859-1")
            df_2008=df_2008.transpose()
            df_2008.reset_index(inplace=True)
            df_2008.columns=[key+'_2008']
        else:
            df_2015=pd.read_csv(dic_all_files[key][year],delimiter=';',nrows=0,encoding="ISO-8859-1")
            df_2015=df_2015.transpose()
            df_2015.reset_index(inplace=True)
            df_2015.columns=[key+'_2015']
        
    Headers_List.append(df_2008.merge(df_2015, left_on=key+'_2008', right_on=key+'_2015', how='outer'))

### Meto cada elemento de la lista en un data frame

In [7]:
#Cabecera del fichero de Accidentes_victimas
Header_Accvict=Headers_List[0]
Header_Accvict

,Accvict_2008,Accvict_2015
0,ID_ACCIDENTE,ID_ACCIDENTE
1,ANIO,ANIO
2,MES,MES
3,HORA,HORA
4,DIASEMANA,DIASEMANA
5,PROVINCIA,PROVINCIA
6,COMUNIDAD_AUTONOMA,COMUNIDAD_AUTONOMA
7,ISLA,ISLA
8,MUNICIPIO,MUNICIPIO
9,TOT_VICTIMAS,TOT_VICTIMAS


In [8]:
#Cabecera del fichero de Vehiculos
Header_Vehic=Headers_List[2]
Header_Vehic

,Vehic_2008,Vehic_2015
0,ID_ACCIDENTE,ID_ACCIDENTE
1,ID_VEHICULO,ID_VEHICULO
2,ANIO_MATRICULA_VEHICULO,ANIO_MATRICULA_VEHICULO
3,MES_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO
4,TIPO_VEHICULO,TIPO_VEHICULO
5,ESTADO_VEHICULO,NaN
6,NUMERO_OCUPANTES_VEH,NUMERO_OCUPANTES_VEH
7,MERCANCIAS_PELIGROSAS,MERCANCIAS_PELIGROSAS
8,VEHICULO_INCENDIADO,VEHICULO_INCENDIADO
9,ANIO,ANIO


In [9]:
#Cabecera del fichero de Personas
Header_Pers=Headers_List[1]
Header_Pers
#Se observa que los datos vienen en columnas con distinto nombre, dependiendo del año


,Pers_2008,Pers_2015
0,ID_ACCIDENTE,ID_ACCIDENTE
1,ID_VEHICULO,ID_VEHICULO
2,ID_PERSONA,ID_PERSONA
3,ID_CONDUCTOR,ID_CONDUCTOR
4,ID_PASAJERO,ID_PASAJERO
5,ID_PEATON,ID_PEATON
6,EDAD,EDAD
7,SEXO,SEXO
8,ANIO_PERMISO,ANIO_PERMISO
9,COD_POSICION,NaN


In [10]:
#Se observa una diferencia en cuanto a nombre y codificacion en los datos de sexo
#entre los datasets anteriores y posteriores a 2011:
    #Para años anteriores a 2011 la codificacion es V o H
    #Para años posteriores, aparece codificados como 1 o 2, por lo que voy a intentar 
    #averiguar cual corresponde a hombre y mujeres
    
#Es logico pensar que en los años anteriores a 1960, por ejwm, el numero de hombres conductores era
#considerablemente al de mujeres, por lo que voy agrupar por edades y año de permiso de circulacion

In [12]:
#Para obtener la agrupacion selecciono los datos mas actuales de los que dispongo 2015
df_pers_sexo=pd.read_csv('./cleaned/MICRODATOS_ACC_VICT_2014/TABLA_PERS_2014.csv',usecols=['EDAD','ID_PERSONA','ANIO_PERMISO','SEXO'],delimiter=";",encoding="ISO-8859-1")
#echo un primer vistazo a los datos
df_pers_sexo.head(5)
df_pers_sexo.tail(5)
df_pers_sexo.info()
#para velores de sexo desconocidos los relleno aleatoriamente
df_pers_sexo[df_pers_sexo['SEXO']==999]=np.random.randint(1,2)

#veo que para las personas que son conductores ('ID_PERSONA'=1), hay valores desconocidos en los años
#de permiso, para no perder esos datos, sustituyo los 9999 por la media del año de permiso de cada edad

#antes convierto la fecha de float a int, para ello necesito sustituir los nulos por 9999 desconocido
df_pers_sexo['ANIO_PERMISO']=df_pers_sexo['ANIO_PERMISO'].fillna(9999)
df_pers_sexo['ANIO_PERMISO']=df_pers_sexo['ANIO_PERMISO'].astype(int)
print ("aqui")
df_pers_sexo.head(35)
#selecciono las personas que son conductores ('ID_PERSONA'=1) y los años desconocidos (9999) que quiero sutituir
pers_grp=df_pers_sexo[(df_pers_sexo['ID_PERSONA']==1) & (df_pers_sexo['ANIO_PERMISO']!=9999)].groupby(['EDAD','SEXO'],as_index=False)['ANIO_PERMISO'].agg({'NUM_CONDUCTORES':'count','MEDIA_ANTIG_CARNET':'mean'})

pers_grp['MEDIA_ANTIG_CARNET']=pers_grp['MEDIA_ANTIG_CARNET'].round(0)
pers_grp['MEDIA_ANTIG_CARNET']=pers_grp['MEDIA_ANTIG_CARNET'].astype(int)

#ahora que tengo la media de año de carnet por edad y sexo, resulvo dos casuisticas:
# 1.la incognita de a que sexo pertenecen los 1 y los 2
# 2.sustituyo los valores 9999 "desconocidos, en eldataframe original por los de las medias
#de segun año y sexo del conductor

df_pers_sexo.columns
pers_grp.columns
#cruzo el df original por edad y sexo con el df con las medias
merged=pd.merge(df_pers_sexo, pers_grp, how='left',on=['EDAD','SEXO'],suffixes=['_left', '_right'])
print("PRE-MODIFICACION")
merged[merged['ANIO_PERMISO']==9999].head(5)

#1. sutituyo los 1 por V y lso 2 por H
merged.ix[(merged['SEXO']==1), 'SEXO'] ='V'
merged.ix[(merged['SEXO']==2), 'SEXO'] ='M'
#2. sustituyo los valores de años de permisos deconocidos o nulos
merged.ix[((merged['ANIO_PERMISO']==9999)) & (df_pers_sexo['ID_PERSONA']==1), 'ANIO_PERMISO'] =merged['MEDIA_ANTIG_CARNET']
print("POST-MODIFICACION")
print ("SOLO TENEMOS AÑOS DESCONOCIDOS PARA NO CONDUCTORES\n")
merged[merged['ANIO_PERMISO']==9999].head(5)
#elimino del df las columnas con el numeor de conductores y la media de año de carnet
merged.drop(['NUM_CONDUCTORES','MEDIA_ANTIG_CARNET'], axis=1, inplace=True)


#pers_grp['ANIO_PERMISO']=pers_grp['ANIO_PERMISO'].astype(int,inplace=True)
#Elimino las columnas que no me aportan informacio
#pers_grp.drop(pers_grp.columns[[0]], axis=1, inplace=True)


,ID_PERSONA,EDAD,SEXO,ANIO_PERMISO
0,1,39,1,2003.0
1,1,52,1,1985.0
2,1,18,1,9999.0
3,1,62,1,1971.0
4,1,25,1,2013.0


,ID_PERSONA,EDAD,SEXO,ANIO_PERMISO
222717,3,11,1,NaN
222718,3,38,1,NaN
222719,3,28,2,NaN
222720,3,20,1,NaN
222721,3,36,2,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222722 entries, 0 to 222721
Data columns (total 4 columns):
ID_PERSONA      222722 non-null int64
EDAD            222722 non-null int64
SEXO            222722 non-null int64
ANIO_PERMISO    154919 non-null float64
dtypes: float64(1), int64(3)
memory usage: 6.8 MB
aqui


,ID_PERSONA,EDAD,SEXO,ANIO_PERMISO
0,1,39,1,2003
1,1,52,1,1985
2,1,18,1,9999
3,1,62,1,1971
4,1,25,1,2013
5,1,29,2,2013
6,1,32,1,9999
7,1,26,2,2008
8,1,62,1,9999
9,1,39,1,9999


Index(['ID_PERSONA', 'EDAD', 'SEXO', 'ANIO_PERMISO'], dtype='object')

Index(['EDAD', 'SEXO', 'NUM_CONDUCTORES', 'MEDIA_ANTIG_CARNET'], dtype='object')

PRE-MODIFICACION


,ID_PERSONA,EDAD,SEXO,ANIO_PERMISO,NUM_CONDUCTORES,MEDIA_ANTIG_CARNET
2,1,18,1,9999,715.0,2013.0
6,1,32,1,9999,2138.0,2005.0
8,1,62,1,9999,841.0,1979.0
9,1,39,1,9999,2327.0,1999.0
11,1,17,1,9999,325.0,2013.0


POST-MODIFICACION
SOLO TENEMOS AÑOS DESCONOCIDOS PARA NO CONDUCTORES



,ID_PERSONA,EDAD,SEXO,ANIO_PERMISO,NUM_CONDUCTORES,MEDIA_ANTIG_CARNET
154919,2,72,V,9999.0,382.0,1974.0
154920,2,35,M,9999.0,1016.0,2003.0
154921,2,22,M,9999.0,703.0,2011.0
154922,2,20,M,9999.0,522.0,2012.0
154923,2,34,V,9999.0,2231.0,2003.0


### Repito el procedimiento anterior para los datos desde 2011 a 2015
### grabando los xls modificados en la carpeta cleaned junto con los otros data set 
### de años 2008 a 2010

In [14]:
!cp -r raw/* cleaned/

In [15]:

person_files_11_15=['./cleaned/MICRODATOS_ACC_VICT_2011/TABLA_PERS_2011.csv',
                    './cleaned/MICRODATOS_ACC_VICT_2012/TABLA_PERS_2012.csv',
                    './cleaned/MICRODATOS_ACC_VICT_2013/TABLA_PERS_2013.csv',
                    './cleaned/MICRODATOS_ACC_VICT_2014/TABLA_PERS_2014.csv',
                    './cleaned/MICRODATOS_ACC_VICT_2015/TABLA_PERS_2015.csv']

def changes_pers_11_15():
    for file in person_files_11_15:
        #leo fichero
        df_pers_sexo=pd.read_csv(file,delimiter=";",encoding="ISO-8859-1")
        #Para sexo desconocido relleno aleatoriamente con 1 o 2
        df_pers_sexo[df_pers_sexo['SEXO']==999]=np.random.randint(1,2)

        #selecciono las personas que son conductores ('ID_PERSONA'=1) y los años desconocidos (9999) que quiero sutituir
        pers_grp=df_pers_sexo[(df_pers_sexo['ID_PERSONA']==1) & (df_pers_sexo['ANIO_PERMISO']!=9999)].groupby(['EDAD','SEXO'],as_index=False)['ANIO_PERMISO'].agg({'NUM_CONDUCTORES':'count','MEDIA_ANTIG_CARNET':'mean'})
        #conversion de a entero
        pers_grp['MEDIA_ANTIG_CARNET']=pers_grp['MEDIA_ANTIG_CARNET'].round(0)
        pers_grp['MEDIA_ANTIG_CARNET']=pers_grp['MEDIA_ANTIG_CARNET'].astype(int)
        
        #cruzo el df original por edad y sexo con el df con las medias
        merged=pd.merge(df_pers_sexo, pers_grp, how='left',on=['EDAD','SEXO'],suffixes=['_left', '_right'])
        merged.head()

        #1. sutituyo los 1 por V y los 2 por H
        merged.ix[(merged['SEXO']==1), 'SEXO'] ='V'
        merged.ix[(merged['SEXO']==2), 'SEXO'] ='M'
        #2. sustituyo los valores de años de permisos deconocidos
        merged.ix[((merged['ANIO_PERMISO']==9999) | (merged['ANIO_PERMISO']==0.0)) & (df_pers_sexo['ID_PERSONA']==1), 'ANIO_PERMISO'] =merged['MEDIA_ANTIG_CARNET']
        merged.head()
        #elimino del df las columnas con el numeor de conductores y la media de año de carnet
        merged.drop(['NUM_CONDUCTORES','MEDIA_ANTIG_CARNET'], axis=1, inplace=True)
        
        #si hubiese algun año NULL, lo relleno a 0.0
        merged['ANIO_PERMISO'].fillna(0.0,inplace=True)
        merged['ANIO_PERMISO']=merged['ANIO_PERMISO'].astype(int)

        #existen errores en los años de permiso, para conductores tenemos valores 0,1 y 999
        #muestro los diferentes valores de año de permiso
        merged['ANIO_PERMISO'].unique()
        #elimino esos registros
        merged=merged[(merged['ANIO_PERMISO'].isin([0,1,999])) & merged['ID_PERSONA']!=1]
        #guardo el fichero
        merged.to_csv(file,index=False,sep=";",encoding="ISO-8859-1")
        print (file+" --- modificado"+"\n")
changes_pers_11_15()

./cleaned/MICRODATOS_ACC_VICT_2011/TABLA_PERS_2011.csv --- modificado

./cleaned/MICRODATOS_ACC_VICT_2012/TABLA_PERS_2012.csv --- modificado

./cleaned/MICRODATOS_ACC_VICT_2013/TABLA_PERS_2013.csv --- modificado

./cleaned/MICRODATOS_ACC_VICT_2014/TABLA_PERS_2014.csv --- modificado

./cleaned/MICRODATOS_ACC_VICT_2015/TABLA_PERS_2015.csv --- modificado



### PARA HOMOGENEIZAR LOS DATOS, MANTEGO SOLO LAS COLUMNAS QUE APARECEN EN TODOS LOS AÑOS
### 1. Datos anteriores a 2010
### 2. Datos posteriores a 2010

In [16]:
Header_Accvict=Header_Accvict[Header_Accvict['Accvict_2008']==Header_Accvict['Accvict_2015']]
Header_Accvict=Header_Accvict['Accvict_2008'].tolist()

Header_Pers=Header_Pers[Header_Pers['Pers_2008']==Header_Pers['Pers_2015']]
Header_Pers=Header_Pers['Pers_2008'].tolist()

Header_Vehic=Header_Vehic[Header_Vehic['Vehic_2008']==Header_Vehic['Vehic_2015']]
Header_Vehic=Header_Vehic['Vehic_2008'].tolist()

dic_all_headers={'Accvict':Header_Accvict,'Pers':Header_Pers,'Vehic':Header_Vehic}
dic_all_headers


{'Accvict': ['ID_ACCIDENTE',
  'ANIO',
  'MES',
  'HORA',
  'DIASEMANA',
  'PROVINCIA',
  'COMUNIDAD_AUTONOMA',
  'ISLA',
  'MUNICIPIO',
  'TOT_VICTIMAS',
  'TOT_MUERTOS',
  'TOT_HERIDOS_GRAVES',
  'TOT_HERIDOS_LEVES',
  'TOT_VEHICULOS_IMPLICADOS',
  'ZONA',
  'ZONA_AGRUPADA',
  'CARRETERA',
  'RED_CARRETERA',
  'TIPO_VIA',
  'TRAZADO_NO_INTERSEC',
  'TIPO_INTERSEC',
  'SUPERFICIE_CALZADA',
  'LUMINOSIDAD',
  'FACTORES_ATMOSFERICOS',
  'VISIBILIDAD_RESTRINGIDA',
  'ACERAS',
  'TIPO_ACCIDENTE'],
 'Pers': ['ID_ACCIDENTE',
  'ID_VEHICULO',
  'ID_PERSONA',
  'ID_CONDUCTOR',
  'ID_PASAJERO',
  'ID_PEATON',
  'EDAD',
  'SEXO',
  'ANIO_PERMISO',
  'MANIOBRAS',
  'INFRACC_VELOCIDAD',
  'INFRACC_PEATON',
  'ANIO'],
 'Vehic': ['ID_ACCIDENTE',
  'ID_VEHICULO',
  'ANIO_MATRICULA_VEHICULO',
  'MES_MATRICULA_VEHICULO',
  'TIPO_VEHICULO',
  'NUMERO_OCUPANTES_VEH',
  'MERCANCIAS_PELIGROSAS',
  'VEHICULO_INCENDIADO',
  'ANIO']}

### ---------------------------------------------------------------------------------------------------PROCEDO A CARGAR EN DATAFRAMES DE CADA UNO DE LOS FICHEROS DEL DICCIONARIO
### ---------------------------------------------------------------------------------------------------

In [17]:
#guardo las claves de cada tipo de fichero
list_keys_dic=dic_all_files.keys()
list_keys_dic

dict_keys(['Accvict', 'Pers', 'Vehic'])

In [18]:
#Creo un dataframe vacio para posteriormente rellenarlo con los años 2001 al 2015
#Recorriendo por cada key los n values del diccionario

dic_total={}
df_total=pd.DataFrame()

for key in list_keys_dic:
    print ("#### "+key+" ####")
    for file in dic_all_files[key]:
        #meto en df cada file quedandome solo con las columnas seleccionadas anteriormente
        df_year=pd.read_csv(file,delimiter=";",encoding="ISO-8859-1",usecols=dic_all_headers[key])
        df_year=df_year.fillna(0)
        print("Cargando fichero-->"+file+"\n\t"+"Registros= "+str(df_year.shape[0])+"\n\t"+"Columnas= "+str(df_year.shape[1]))
        #df_total=df_total.append(df_year)
        df_total=pd.concat([df_total,df_year],ignore_index=True)
    dic_total[key] = df_total
print ("\n\t\t"+"##### CARGADOS EN DF_TOTAL TODOS LOS PERIODOS ########"+"\t\t")
df_total.shape

#### Accvict ####
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv
	Registros= 93161
	Columnas= 27
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv
	Registros= 88251
	Columnas= 27
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv
	Registros= 85503
	Columnas= 27
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv
	Registros= 83027
	Columnas= 27
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv
	Registros= 83115
	Columnas= 27
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv
	Registros= 89519
	Columnas= 27
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv
	Registros= 91570
	Columnas= 27
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv
	Registros= 97756
	Columnas= 27
#### Pers ####
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_PERS_24H_2008.csv
	Regi

(3585277, 44)

In [19]:
dic_total['Accvict'].info()
dic_total['Accvict'].describe(include = 'all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711902 entries, 0 to 711901
Data columns (total 27 columns):
ID_ACCIDENTE                711902 non-null object
ANIO                        711902 non-null int64
MES                         711902 non-null int64
HORA                        711902 non-null int64
DIASEMANA                   711902 non-null int64
PROVINCIA                   711902 non-null int64
COMUNIDAD_AUTONOMA          711902 non-null int64
ISLA                        711902 non-null float64
MUNICIPIO                   711902 non-null float64
TOT_VICTIMAS                711902 non-null int64
TOT_MUERTOS                 711902 non-null int64
TOT_HERIDOS_GRAVES          711902 non-null int64
TOT_HERIDOS_LEVES           711902 non-null int64
TOT_VEHICULOS_IMPLICADOS    711902 non-null int64
ZONA                        711902 non-null int64
ZONA_AGRUPADA               711902 non-null int64
CARRETERA                   711902 non-null object
RED_CARRETERA               71190

,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,...,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
count,7.119020e+05,711902.000000,711902.000000,711902.000000,711902.00000,711902.000000,711902.000000,711902.000000,711902.000000,711902.000000,...,711902.000000,711902.000000,711902.000000,711902.000000,711902.000000,711902.000000,711902.000000,711902.000000,711902.0,711902.000000
unique,4.553670e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN
top,2.009080e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,998.0,NaN
freq,1.836400e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,385762.0,NaN
mean,NaN,2011.542770,6.560167,13.896996,3.89311,23.436101,8.542552,11.816708,18300.746423,1.419988,...,3.289513,6.206565,44.287391,0.823089,6.104594,1.639012,16.290773,362.663112,NaN,15.858746
std,NaN,2.339568,3.403688,5.367769,1.94614,14.437490,4.520992,60.213706,15932.298446,0.922946,...,1.119739,2.627062,203.721735,1.739577,68.352016,1.083758,120.339702,478.127357,NaN,17.941449
min,NaN,2008.000000,1.000000,0.000000,1.00000,1.000000,1.000000,0.000000,0.000000,1.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,NaN,1.000000
25%,NaN,2009.000000,4.000000,10.000000,2.00000,8.000000,5.000000,0.000000,3099.000000,1.000000,...,2.000000,3.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,NaN,4.000000
50%,NaN,2012.000000,7.000000,14.000000,4.00000,25.000000,9.000000,0.000000,13039.000000,1.000000,...,4.000000,8.000000,1.000000,0.000000,1.000000,1.000000,1.000000,8.000000,NaN,12.000000
75%,NaN,2014.000000,10.000000,18.000000,5.00000,35.000000,12.000000,0.000000,29015.000000,2.000000,...,4.000000,8.000000,1.000000,2.000000,1.000000,2.000000,1.000000,999.000000,NaN,18.000000


#### AHORA TENGO DE DATA FRAMES CON TODOS LOS AÑOS DESDE 2008 HASTA 2015

In [56]:
print ("Años disponible DF de Acidentes\n")
dic_total['Accvict']['ANIO'].unique()
print ("Años disponible DF de personas\n")
dic_total['Pers']['ANIO'].unique()
print ("Años disponible DF de vehiculos\n")
dic_total['Vehic']['ANIO'].unique()


array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015])

array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015])

array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015])

,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,...,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
0,2008480000372,2008,4,7,5,48,17,0.0,0.0,4,...,3,2,1.0,0.0,2,3,1,8,N,71
1,2008250021218,2008,3,14,7,25,9,0.0,0.0,1,...,2,5,2.0,0.0,1,1,9,8,N,22
2,2008430024095,2008,10,12,7,43,9,0.0,0.0,1,...,4,5,1.0,0.0,1,1,1,0,N,68
3,2008430022973,2008,4,18,2,43,9,0.0,0.0,1,...,4,5,0.0,3.0,1,1,1,0,N,14
4,2008250021457,2008,5,19,5,25,9,0.0,0.0,1,...,4,9,1.0,0.0,9,1,9,8,N,66


,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,...,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
711897,2016990000448,2015,12,13,2,28,13,0.0,28079.0,1,...,4,8,0.0,2.0,1,1,1,999,999,7
711898,2016990000873,2015,12,7,4,28,13,0.0,28079.0,1,...,4,8,0.0,2.0,3,3,4,999,998,2
711899,2016990000875,2015,12,11,4,28,13,0.0,28079.0,2,...,4,8,0.0,2.0,3,1,4,999,998,2
711900,2016990000876,2015,12,13,2,28,13,0.0,28079.0,1,...,4,8,1.0,0.0,1,1,1,999,999,7
711901,2016990001660,2015,11,15,1,28,13,0.0,28079.0,1,...,4,8,0.0,2.0,1,1,1,999,998,2


In [ ]:

dic_total['Accvict'].head()

dic_total['Accvict'].tail()

In [92]:
##para categorizar
#dic_total['Accvict']['HORA']=dic_total['Accvict']['HORA'].astype('category')
#dic_total['Accvict']['HORA'].describe()
#este funciona
dic_total['Accvict']['HORA']= pd.Series(pd.Categorical(dic_total['Accvict']['HORA'], categories=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]))
dic_total['Accvict']['HORA'].cat.rename_categories(['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23'])
dic_total['Accvict']['HORA'].cat.codes

count     711902
unique        24
top           14
freq       50900
Name: HORA, dtype: int64

0          7
1         14
2         12
3         18
4         19
5         14
6         20
7         19
8          2
9         13
10        22
11        21
12        13
13        18
14        18
15         8
16        17
17        15
18        14
19        11
20         0
21        19
22        18
23         8
24        10
25        20
26        21
27        17
28         6
29         1
          ..
711872    16
711873     9
711874    10
711875     7
711876     9
711877     8
711878    20
711879     7
711880    11
711881     0
711882    13
711883    13
711884    12
711885    12
711886    12
711887    18
711888     9
711889     0
711890     1
711891    13
711892    16
711893    13
711894    14
711895    11
711896    12
711897    13
711898     7
711899    11
711900    13
711901    15
dtype: category
Categories (24, object): [0, 1, 2, 3, ..., 20, 21, 22, 23]

0          7
1         14
2         12
3         18
4         19
5         14
6         20
7         19
8          2
9         13
10        22
11        21
12        13
13        18
14        18
15         8
16        17
17        15
18        14
19        11
20         0
21        19
22        18
23         8
24        10
25        20
26        21
27        17
28         6
29         1
          ..
711872    16
711873     9
711874    10
711875     7
711876     9
711877     8
711878    20
711879     7
711880    11
711881     0
711882    13
711883    13
711884    12
711885    12
711886    12
711887    18
711888     9
711889     0
711890     1
711891    13
711892    16
711893    13
711894    14
711895    11
711896    12
711897    13
711898     7
711899    11
711900    13
711901    15
Name: HORA, dtype: category
Categories (24, int64): [0, 1, 2, 3, ..., 20, 21, 22, 23]

0          7
1         14
2         12
3         18
4         19
5         14
6         20
7         19
8          2
9         13
10        22
11        21
12        13
13        18
14        18
15         8
16        17
17        15
18        14
19        11
20         0
21        19
22        18
23         8
24        10
25        20
26        21
27        17
28         6
29         1
          ..
711872    16
711873     9
711874    10
711875     7
711876     9
711877     8
711878    20
711879     7
711880    11
711881     0
711882    13
711883    13
711884    12
711885    12
711886    12
711887    18
711888     9
711889     0
711890     1
711891    13
711892    16
711893    13
711894    14
711895    11
711896    12
711897    13
711898     7
711899    11
711900    13
711901    15
dtype: int8

,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,...,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
0,"2,01E+12",2008,4,7,5,48,17,0.0,0.0,4,...,3,2,1.0,0.0,2,3,1,8,N,71
1,"2,01E+12",2008,3,14,7,25,9,0.0,0.0,1,...,2,5,2.0,0.0,1,1,9,8,N,22
2,"2,01E+12",2008,10,12,7,43,9,0.0,0.0,1,...,4,5,1.0,0.0,1,1,1,0,N,68
3,"2,01E+12",2008,4,18,2,43,9,0.0,0.0,1,...,4,5,0.0,3.0,1,1,1,0,N,14
4,"2,01E+12",2008,5,19,5,25,9,0.0,0.0,1,...,4,9,1.0,0.0,9,1,9,8,N,66


,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,...,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
711897,2016990000448,2015,12,13,2,28,13,0.0,28079.0,1,...,4,8,0.0,2.0,1,1,1,999,999,7
711898,2016990000873,2015,12,7,4,28,13,0.0,28079.0,1,...,4,8,0.0,2.0,3,3,4,999,998,2
711899,2016990000875,2015,12,11,4,28,13,0.0,28079.0,2,...,4,8,0.0,2.0,3,1,4,999,998,2
711900,2016990000876,2015,12,13,2,28,13,0.0,28079.0,1,...,4,8,1.0,0.0,1,1,1,999,999,7
711901,2016990001660,2015,11,15,1,28,13,0.0,28079.0,1,...,4,8,0.0,2.0,1,1,1,999,998,2


In [22]:
#Extraigo los datos cuantitativos de victimas por
# 1. Año-mes-dia de la semana
# 2. Localizacion geografica

accvict_geo=pd.get_dummies(dic_total['Accvict'], prefix=['CA','PROV'], columns=['COMUNIDAD_AUTONOMA','PROVINCIA'])


In [23]:
dic_total['Accvict']

,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,...,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
0,2008480000372,2008,4,07:00,5,48,17,0.0,0.0,4,...,3,2,1.0,0.0,2,3,1,8,N,71
1,2008250021218,2008,3,14:00,7,25,9,0.0,0.0,1,...,2,5,2.0,0.0,1,1,9,8,N,22
2,2008430024095,2008,10,12:00,7,43,9,0.0,0.0,1,...,4,5,1.0,0.0,1,1,1,0,N,68
3,2008430022973,2008,4,18:00,2,43,9,0.0,0.0,1,...,4,5,0.0,3.0,1,1,1,0,N,14
4,2008250021457,2008,5,19:00,5,25,9,0.0,0.0,1,...,4,9,1.0,0.0,9,1,9,8,N,66
5,2008250022083,2008,12,14:00,1,25,9,0.0,0.0,3,...,2,5,2.0,0.0,1,1,1,8,N,67
6,2008430024521,2008,12,20:00,2,43,9,0.0,0.0,1,...,4,9,1.0,0.0,1,5,1,0,N,12
7,2008480000579,2008,6,19:00,5,48,17,0.0,0.0,2,...,5,1,1.0,0.0,1,1,1,8,N,13
8,2008430022598,2008,3,02:00,7,43,9,0.0,0.0,2,...,4,5,0.0,1.0,1,5,1,0,N,68
9,2008080018378,2008,12,13:00,2,8,9,0.0,0.0,1,...,4,5,0.0,3.0,9,1,1,0,N,41


In [24]:
df = pd.DataFrame({'A': ['x', 'y', 'x'], 'B': ['z', 'u', 'z'],
                  'C': [1,2,3],
                  'D':['j', 'l', 'j']})

In [25]:
df

,A,B,C,D
0,x,z,1,j
1,y,u,2,l
2,x,z,3,j


In [26]:
df_1= pd.get_dummies(df)
df_1

,C,A_x,A_y,B_u,B_z,D_j,D_l
0,1,1,0,0,1,1,0
1,2,0,1,1,0,0,1
2,3,1,0,0,1,1,0


In [27]:
df_2=pd.get_dummies(df, prefix=['DDDD','DDD','D'], columns=['A','B','C'])
df_2

,D,DDDD_x,DDDD_y,DDD_u,DDD_z,D_1,D_2,D_3
0,j,1,0,0,1,1,0,0
1,l,0,1,1,0,0,1,0
2,j,1,0,0,1,0,0,1


In [28]:
  df_year=pd.read_csv(files_list[file],delimiter=";",encoding="ISO-8859-1",converters={'COD_MUNICIPIO': str, 'MUNICIPIO': str})

NameError: name 'files_list' is not defined


### LECTURA DE LOS DATASET DE VEHICULOS PARA LOS AÑOS 2011-2015

In [ ]:
df_year.shape[0]

In [187]:
#Creo un dataframe vacio para posteriormente rellenarlo con los años 2001 al 2015
df_accvict_08_15 = pd.DataFrame()
dict={}
for key in list_keys_dic:
    for file in range(0,len(files_list)):
        df_year=pd.read_csv(files_list[file],delimiter=";",encoding="ISO-8859-1")
        df_year.fillna(0)
        dict[tipology]=files_list[file]
        print("Cargando fichero-->"+files_list[file]+"\n\t"+"Registros= "+str(df_year.shape[0])+"\n\t"+"Columnas= "+str(df_year.shape[1]))
        if tipology=='list_accvict'
        %(tipology)+_08_15=df_accvict_08_15.append(df_year)


Cargando fichero-->./MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv
	Registros= 93161
	Columnas= 32
Cargando fichero-->./MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv
	Registros= 88251
	Columnas= 32
Cargando fichero-->./MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv
	Registros= 85503
	Columnas= 32
Cargando fichero-->./MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv
	Registros= 83027
	Columnas= 39
Cargando fichero-->./MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv
	Registros= 83115
	Columnas= 39
Cargando fichero-->./MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv
	Registros= 89519
	Columnas= 39
Cargando fichero-->./MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv
	Registros= 91570
	Columnas= 39
Cargando fichero-->./MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv
	Registros= 97756
	Columnas= 39


In [189]:
j=df_accvict_08_15.groupby(['ANIO']).size()


In [191]:
type(j)

pandas.core.series.Series

In [ ]:
df_accvict_08_15.groupby(['ANIO']).agg(['mean', 'count'])

In [176]:
print(df_accvict_08_15.head())
#df1=df[['ANIO','TOT_VICTIMAS']]


  ACERAS  ACOND_CALZADA  ANIO CARRETERA  COD_MUNICIPIO  COMUNIDAD_AUTONOMA  \
0      N            0.0  2008    BI-631            NaN                  17   
1      N            0.0  2008      C-12            NaN                   9   
2      N            0.0  2008   TV-7331            NaN                   9   
3      N            0.0  2008     N-340            NaN                   9   
4      N            0.0  2008     CR-26            NaN                   9   

   DENSIDAD_CIRCULACION  DIASEMANA  FACTORES_ATMOSFERICOS   HORA  \
0                   1.0          5                      1  07:00   
1                   1.0          7                      9  14:00   
2                   0.0          7                      1  12:00   
3                   0.0          2                      1  18:00   
4                   1.0          5                      9  19:00   

       ...        TOT_HERIDOS_LEVES30D  TOT_MUERTOS  TOT_MUERTOS30D  \
0      ...                         NaN            0

In [174]:
dict

{0: './MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv',
 1: './MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv',
 2: './MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv',
 3: './MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv',
 4: './MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv',
 5: './MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv',
 6: './MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv',
 7: './MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv'}

In [ ]:
threedee.scatter(df.ANIO, df["TOT_MUERTOS"], df['Volume'])